<h1 align = "center"> Home Value Predictions in Greater Boston Area </h1> 

<img src="static/img/mls_listing.png" width=800 align="center">

<h6 align = "center"> MLS Listing Preview </h6> 

## Introduction

A home is often the largest and most expensive purchase a person makes in his or her lifetime. Ensuring homeowners have a trusted way to monitor this asset is incredibly important. While many individual homebuyers are less sensitive to price forecasts, small errors in price prediction can have systemic negative effects in the economy as a whole. Accurate prediction makes it easier to understand which features would influence the final property price.

## Problem Statement

Real estate prices are very much dependent on factors that are not easy to control. Analyzing broader market conditions and specific property determinants in order to establish how property values may change over the course of time are utterly important. Massive data can be obtained about the current market situation, which demands using powerful machine learning algorithms in order to predict with high precision and in a reasonable time frame.

*Goal: Explore different models using internal data source (MLS data). Choose the best model as the final home valuation algorithms to predict prices on a new set of listings. Use price predictions and other listing features to build the app for the "Midterm_Part_II".*

### Load libraries

In [ ]:
import pandas as pd
import numpy as np

### Load Data

Before we start with modelling, we have to take a closer look at our data features:

In [ ]:
# Read dataset columns
all_columns = pd.read_csv('static/data/sold_geocoded.csv').columns.values
all_columns

Clearly, there are too many features in the dataset. We don't need all of them for this task. Let us choose a good set of predictors. 

In [ ]:
use_cols = ['MLSNUM','DisplayX','DisplayY','ZIP','AGE','BATHS','BEDS','GARAGE','LISTDATE',
            'LISTPRICE','LOTSIZE','PROPTYPE','SOLDPRICE','SQFT', 'PPSF', 'PHOTOURL', 'REMARKS']

Data description:

    1. MLSNUM - listing number
    2. DisplayX - latitude
    3. DisplayY - longitude
    4. ZIP - ZIP code
    5. AGE - age of the building
    6. BATHS - number of bathrooms 
    7. BEDS - number of bedrooms 
    8. GARAGE - number of parking spots in the building
    9. LISTDATE - listed date of sale
    10. LISTPRICE - asking price
    11. LOTSIZE - building lotsize
    12. PROPTYPE - building type (CC-condo; MF-multifamily, SF-single family)
    13. SOLDPRICE - condo final price
    14. SQFT - total condo square feet
    15. PPSF - price per square foot
    16. PHOTOURL - single condo picture URL
    17. REMARKS - descriptions

In [ ]:
# Load data with selected set of features
df_all = pd.read_csv('static/data/sold_geocoded.csv', usecols=use_cols)
df_all.head()

### Data Preprocessing

This section includes preprocessing, technique necessary for the raw data transformation into an understandable format. MLS data may be often incomplete, inconsistent, and/or lacking in certain infomration, and is likely to contain many errors. Data preprocessing is a proven method of resolving such issues.

In [ ]:
# Clean PPSF data
# PPSF has many "inf" values that need to be removed
# Take a look at the function below and 
# make sure there are no any "nan" or "inf" values left

df_all['PPSF'] = df_all['PPSF'].round(2)

# array to store indices
indexes = []

# colect all indices that contain "inf" value
for p in enumerate(df_all['PPSF']):
    if p[1] == np.inf:
        indexes.append(p[0])
        
# drop all rows with "inf" values        
df_all = df_all.drop(df_all.index[indexes])  

In [ ]:
# Check the shape of the data - you should be able to see (158874, 17)
df_all.shape

YOUR TURN <br><br>
**Question 1 (1 pt):** <br> *Create new column 'LISTMONTH', extracting the information from the 'LISTDATE' feature.*

In [ ]:
# YOUR CODE HERE




In [ ]:
# Check datatypes
df_all.dtypes

The above output should be exactly the same as this one:<br>
<img src="static/img/dtypes.png" align="left">

YOUR TURN <br> <br> **Question 2 (1 pt):** <br> *One-hot encode 'ZIP' feature, using pandas get_dummies() method.*

In [ ]:
# YOUR CODE HERE





In [ ]:
# The final cleaning step is to take care of missing values
# Drop all "nan" rows/cols using .dropna() pandas method
df_all.dropna(inplace=True)

In [ ]:
# Final check for "nan" values
# Notice the use of .head(18) here!!!
# We are preventing long outputs and
# try to print only columns that corespond to "use_cols" features set
# Important: Remove .head(18) to check all features

df_all.isnull().sum().head(18)

YOUR TURN <br><br>
**Question 3 (1 pt):** <br> *Extract condos data and create the new dataframe called "condos".* 
<br><br>Hint: PROPTYPE has 3 unique values: 'CC','MF', and 'CC', where 'CC' stands for **condos**.

In [ ]:
# YOUR CODE HERE




Let us sort all condo values by 'LISTDATE' and check the final shape of the data:

In [ ]:
# Sort all values by 'LISTDATE'
condos = condos.sort_values(by='LISTDATE')
condos.head()

In [ ]:
# Condos shape - you shoud see (10027, 725)
condos.shape

### Split data into "future_data" and "training_data"

So far we have worked on cleaning the data. Now, we want to split it into two datasets: "training_data" and "future_data". The first dataset ("training_data") as its name implies will serve for training and second dataset  ("future_data") will act like a future, "unseen" data. The final (best) model will be used to predict prices on "future_data" dataset.

In [ ]:
# Create "future_data" dataset
# Future data does not have SOLDPRICE and thus, the feature should be excluded
# NOTE: Future data will have 60 listings - see how we slice the data 
# Export data as "midterm_future_data.csv", storing the dataset in your "data" folder

future_data = condos[9967:]
future_data_features = list(set(future_data.columns) - set(['SOLDPRICE'])) 
future_data = future_data[future_data_features]
future_data.to_csv('static/data/midterm_future_data.csv', index=None)

In [ ]:
# Check future_data shape - you shoud see (60, 724)
future_data.shape

YOUR TURN <br><br>
**Question 4 (1 pt):** <br> *Create "training_data.csv" dataset and store it into your "data" folder.* <br><br>Training data should contain all data except the last 60 obsevrations we have periously exported as "midterm_future_data.csv" data. 

In [ ]:
# YOUR CODE HERE





In [ ]:
# Check the shape of your data - you shoud see (9967, 725)
training_data.shape

Before we move on, make sure you have "midterm_future_data.csv" and "midterm_training_data.csv" datasets stored into your "data" folder (they will be together with you "sold_geocoded.csv" dataset):

<img src="static/img/datasets.png" align="left">

## Data Modeling

In this section we are going to train, test, and choose the best performing model. The steps are the following:
    1. Load "midterm_training_data.csv" 
    2. Choose an appropriate feature set
    3. Train/Test data by using 2 models:
        - Random Forest
        - Gradient Boosting
    4. Choose better model and export predictions

#### Load "midterm_training_data.csv" dataset

In [ ]:
# Load data
train = pd.read_csv('static/data/midterm_training_data.csv')
train.head()

#### Select features

Exclude 'LISTPRICE','SOLDPRICE', and 'LISTDATE' given that, these are unknown features for any given listing in the future. 'DisplayX'and 'DisplayY' cannot be used as predictors. Usually, they have been used to create distance matrices necessary for estimating the nearest neighbors, which is outside of the scope of this work. 'PHOTOURL' are URL links and 'MLSNUM' is a Database ID. 'REMARKS' are descritpions (we are not doing text analysis here) and thus we are exclusing them as well. We have already extracted condos ('CC') from the 'PROPTYPE' feature.

In [ ]:
# Create train_features valiable and store selected features in it
train_features = list(set(train.columns) - set(['PHOTOURL','MLSNUM','SOLDPRICE','REMARKS','LISTDATE',
                                          'LISTPRICE','PROPTYPE', 'DisplayX','DisplayY']))

#### Train/Test data

In [ ]:
# Load sklearn library
from sklearn.model_selection import train_test_split

YOUR TURN <br><br>
**Question 5 (1 pt):** <br> *Train - Test - **Split** * <br><br>Use sklearn to split the train dataset into train and test. Specifically, create 'X_train', 'X_test', 'y_train', and 'y_test' datasets. Use 70% for the train and 30% for the test data. Come up with your unique random_state number."  

In [ ]:
# YOUR CODE





In [ ]:
# Check the shape of the data
# You should get: 
# X_train.shape = (6976, 716), y_train.shape = (6976,)
# X_test.shape = (2991, 716), y_test.shape = (2991,)

print('X_train.shape = {}, y_train.shape = {}'.format(X_train.shape, y_train.shape))
print('X_test.shape = {}, y_test.shape = {}'.format(X_test.shape, y_test.shape))

### Training Your Models

In [ ]:
# Load library
from sklearn.ensemble import RandomForestRegressor

YOUR TURN <br><br>
**Question 6 (1 pt):** <br> *Train data using Random Forest model.*

In [ ]:
# YOUR CODE HERE





In [ ]:
# Load library
from sklearn.ensemble import GradientBoostingRegressor

YOUR TURN <br><br>
**Question 7 (1 pt):** <br> *Train data using Gradient Boosting model.*

In [ ]:
# YOUR CODE HERE




### Predict on Future Data

In [ ]:
# Import "future_data" dataset
future_data = pd.read_csv("static/data/midterm_future_data.csv")
future_data.head()  

In [ ]:
# Print future_data columns 
# You can use future_data.columns.values 

future_data.columns

We want to extract only valid predictors from "future_data" dataset. Note: we don't want to use features such as 'PHOTOURL','MLSNUM' or 'LISTPRICE' as our predictors.

YOUR TURN <br><br>
**Question 8 (1 pt):** <br> Create "predicion_features" variable to store the approprate set of features. <br>Note: The number of features needs to coresspond to the features set used for training. 

In [ ]:
# YOUR CODE HERE





In [ ]:
# Create new dataset called "predictions_df"
# The dataset will contain only "predicion_features" set

predictions_df = future_data[predicion_features]

YOUR TURN <br><br>
**Question 9 (1 pt):** <br> Predict prices using your best model. Store predictions in the variable called "predictions". 

In [ ]:
# YOUR CODE HERE




In [ ]:
# Round prediction values 
predictions = predictions.round(2)

In [ ]:
# Add new column 'predicted_price' to the 'future_data' dataset
# Pass newly predicted values

future_data['predicted_price'] = predictions

In [ ]:
# Choose the final set of columns to be use for you final export

final_cols = ['DisplayY','DisplayX','BEDS','SQFT','PHOTOURL','PPSF',
            'BATHS','MLSNUM','LISTPRICE', 'REMARKS', 'predicted_price']

In [ ]:
# Create new dataframe for export
# NOTE: This dataframe will be exported to csv file and used in Midterm_Part_II

for_export_df = future_data[final_cols]

In [ ]:
# Change column names (small caps)
for_export_df = for_export_df.rename(columns={'REMARKS':'remarks', 'LISTPRICE':'list_price', 'DisplayY':'display_y', 
                                        'DisplayX':'display_x', 'BEDS':'beds', 'SQFT':'sqft', 'PHOTOURL':'photo_url', 
                                        'PPSF':'ppsf', 'BATHS':'baths', 'MLSNUM':'mlsnum'})

In [ ]:
# Round lon/lat data
# Parse MLS number to int 
# Note: For MLS number, you may want to use string later, in the Midterm_Part_II 

for_export_df['display_y'] = for_export_df['display_y'].round(5)
for_export_df['display_x'] = for_export_df['display_x'].round(5)
for_export_df['mlsnum'] = for_export_df['mlsnum'].astype(int)

In [ ]:
# Final check
for_export_df.head()

Your dataset (from above) should look like this (**'predicted_price' figures may be different due to dfferent splits and that is ok! **):<br>
<img src="static/img/final_dataset.png" align="center">

YOUR TURN <br><br>
**Question 10 (1 pt):** <br> Export "for_export_df" dataframe into "midterm.csv" file and store it into your "data" folder. You will model the "condos" database table according to it, so make sure you have completed this step!

In [ ]:
# YOUR CODE HERE





In [ ]:
# Check the dataset shape - you should see (60, 11)
for_export_df.shape

Your "data" folder should look like this (together with "sold_geocoded.csv"):

<img src="static/img/data2.png" align="left">

Congratulations! You finished the Part I. :)